In [5]:
# 7章　仮説と推定

# 7.2

import math

def normal_approximation_to_binomial(n,p):
    """Binomial(n,p)に相当するμとσを計算する"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

def normal_cdf(x, mu=0, sigma=1):
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

def inverse_normal_cdf(p, mu=0, sigma=1, tolerance=0.00001):
    if mu != 0 or sigma != 1:
        return mu + sigma * inverse_normal_cdf(p, tolerance=tolerance)
    
    low_z, low_p = -10.0, 0
    hi_z, hi_p = 10.0, 1
    while hi_z - low_z > tolerance:
        mid_z = (low_z + hi_z) / 2
        mid_p = normal_cdf(mid_z)
        if mid_p < p:
            low_z, low_p = mid_z, mid_p
        elif mid_p > p:
            hi_z, hi_p = mid_z, mid_p
        else:
            break
    return mid_z

# 変数が閾値を下回る確率はnormal_cdfで表せる
normal_probability_below = normal_cdf

# 閾値を下回っていなければ、閾値より上にある
def normal_probability_above(lo, mu=0, sigma=1):
    return 1 - normal_cdf(lo, mu, sigma)

# hiより小さく、loより大きければ、値はその間にある
def normal_probability_between(lo, hi, mu=0, sigma=1):
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# 間に無ければ、範囲外にある
def normal_probability_outside(lo, hi, mu=0, sigma=1):
    return 1 - normal_probability_between(lo, hi, mu, sigma)

# 同じことは逆の手順でも可能。
# 例えば平均を中心として60％の確率で発生する領域を求めたければ、上下それぞれの確率が20%の分を取り除けばよい
def normal_upper_bound(probability, mu=0, sigma=1):
    """確率P(Z<=z)となるzを返す"""
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability, mu=0, sigma=1):
    """確率P(Z>=z)となるzを返す"""
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability, mu=0, sigma=1):
    """指定された確率を包含する（平均を中心に）対称な境界を返す"""
    tail_probability = (1 - probability) / 2
    
    # 上側の境界はテイル確率（tail_probability）分、上に
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    
    # 下側の境界はテイル確率（tail_probability）分、下に
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    
    return lower_bound, upper_bound



In [7]:
# コイン投げの回数を1000回として、コインに歪みはない（p=0.5）という仮説が真であるならば、
# Xは平均が500で分散15.8の正規分布で近似できる。

mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)
print(mu_0, sigma_0)

# 第一種の過誤（偽陽性）をどの程度受け入れるか、いわゆる有意性を決める。ここでは5%とする

ans = normal_two_sided_bounds(0.95, mu_0, sigma_0)
print(ans)

500.0 15.811388300841896


In [11]:
# 第二種の過誤が起きない確率について考える。
# コインが少しだけ歪んでいて、p=0.55だとしたら？

# pが0.5であると想定の元で、95%の境界を確認する
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)
print(lo, hi)

# p=0.55であった場合の、μとσを計算する
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)
print(mu_1, sigma_1)

# 第二種過誤とは、帰無仮説を棄却しないという誤りがあり、
# Xが当初想定の領域に入っている場合に生じる
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability
print(power)

469.01026640487555 530.9897335951244
550.0 15.732132722552274
0.8865480012953671
